# 0. Important Notes
This notebook was created for the project "Just Sign" which is able to detect various signing motions for the song "Cupid" by FIFTY FIFTY.

Created with the help of Nicholas Renotte's tutorial on action recogniton [here](https://www.youtube.com/watch?v=doDUihpj6ro&t=2862s&ab_channel=NicholasRenotte) as well as ChatGPT's responses on integrating variable-length input in real-time scenarios.

# 1. Installing and Importing Dependencies

In [ ]:
!pip install mediapipe tensorflow sklearn opencv-python scikit-learn

In [3]:
import os
import cv2
import time

import numpy as np
import mediapipe as mp

# 2. Setup Folders for Collection

In [54]:
# Path for exported data
DATA_PATH = os.path.join("Training Data")

# Array of all actions to detect
all_actions = np.array([
#                         "HOPELESS",     # START HALF ONE
#                         "SWEETHEART",  
#                         "ALL", 
#                         "ME", 
#                         "LIFE", 
#                         "COUPLE", 
#                         "SURROUND", 
#                         "ALL", 
#                         "TIME", 
#                         "I", 
#                         "GUESS",  
#                         "MEANS", 
#                         "SOMETHING", 
#                         "WHY", 
#                         "FEEL", 
#                         "LONELY", 
#                         "WISH", 
#                         "FIND", 
#                         "LOVER", 
#                         "HUG", 
#                         "NOW",           # START HALF TWO
#                         "CRY", 
#                         "ROOM", 
#                         "SKEPTICAL",  
#                         "LOVE", 
#                         "BUT", 
#                         "STILL", 
#                         "MORE", 
#                         "GIVE", 
#                         "NEW", 
#                         "CHANCE",  
#                         "CUPID", 
#                         "STUPID", 
#                         "HE", 
#                         "MAKE", 
#                         "THAT", 
#                         "NOT", 
#                         "REAL", 
#                         "DUMB", 
                        "NO_DETECTIONS" 
                        ])

# Each action has 35 videos consisting of 25 frames
num_sequences = 35
sequence_length = 25

In [55]:
# Create directories for each action
for action in all_actions:
    for sequence in range(num_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            
        except:
            pass

# 3. Collect Keypoints Sequences

In [6]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [7]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    results = model.process(image)
    
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    return image, results

In [8]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image,
                              results.face_landmarks,
                              mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(192, 255, 48), thickness=1, circle_radius=1), 
                              mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=1, circle_radius=1))
    
    # Draw pose connections
    mp_drawing.draw_landmarks(image,
                              results.pose_landmarks,
                              mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(192, 255, 48), thickness=2, circle_radius=3), 
                              mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2))
    
    # Draw left hand connections
    mp_drawing.draw_landmarks(image,
                              results.left_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(192, 255, 48), thickness=2, circle_radius=3), 
                              mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2))
    
    # Draw right hand connections
    mp_drawing.draw_landmarks(image,
                              results.right_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(192, 101, 21), thickness=2, circle_radius=3), 
                              mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2))

In [9]:
def extract_keypoints(results):
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([face, pose, left_hand, right_hand])

In [ ]:
video_capture = cv2.VideoCapture(0)

# Setup the MediaPipe model
with mp_holistic.Holistic(min_detection_confidence=0.75, min_tracking_confidence=0.75) as holistic:
    # Loop through all the actions
    for action in all_actions:
        # Loop through all the videos
        for sequence in range(num_sequences):
            # Loop through all the frames
            for frame_num in range(sequence_length):
                # Read video feed
                success, frame = video_capture.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Apply wait logic for recording
                if frame_num == 0:
                    cv2.putText(image, "STARTING COLLECTION", (120, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)

                    cv2.putText(image, f"Current Action: {action} - Video Number {sequence}", (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    # Show to screen
                    cv2.imshow("OpenCV Feed", image)
                    cv2.waitKey(1500)
                    
                else:
                    cv2.putText(image, f"Current Action: {action} - Video Number {sequence}", (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    # Show to screen
                    cv2.imshow("OpenCV Feed", image)
                      
                # Save the keypoints
                keypoints = extract_keypoints(results)
                np_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(np_path, keypoints)
                
                # Quit application
                if cv2.waitKey(10) & 0xFF == ord("q"):
                    break
            
    video_capture.release()
    cv2.destroyAllWindows()

# 4. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(all_actions)}

In [ ]:
labels, sequences = [], []

for action in all_actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        video = []
        
        for frame_num in range(sequence_length):
            frame = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            video.append(frame)
            
        sequences.append(video)
        labels.append(label_map[action])

In [ ]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

# 5. Build and Train LSTM Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.backend import ctc_batch_cost
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
model = Sequential()

model.add(LSTM(64, return_sequences=True, activation="relu", input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation="relu"))
model.add(LSTM(64, return_sequences=False, activation="relu"))

model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(all_actions.shape[0], activation="softmax"))

In [ ]:
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=["categorical_accuracy"])

In [ ]:
model.fit(x_train, y_train, epochs=275, callbacks=[Tensorboard(log_dir=os.path.join("TensorBoard Logs"))])

In [ ]:
model.save("ASL Recognition Model.h5")

# 6. Make Real-Time Predictions

In [ ]:
sequence = []
predictions = []
threshold = 0.5

video_capture = cv2.VideoCapture(0)
model = tf.keras.models.load_model("ASL Recognition Model.h5")

# Setup the MediaPipe model
with mp_holistic.Holistic(min_detection_confidence=0.75, min_tracking_confidence=0.75) as holistic:
    while video_capture.isOpened():
        # Read video feed
        success, frame = video_capture.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Make predictions
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-25:]
        
        if len(sequence) == 25:
            result = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(all_actions[np.argmax(result)])
            predictions.append(np.argmax(result))
        
        # Show to screen
        cv2.imshow("OpenCV Feed", image)

        # Quit application
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
            
    video_capture.release()
    cv2.destroyAllWindows()